In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import random
import pandas as pd
import xlsxwriter

In [1]:
id_ = input('아이디를 입력하세요:   ')
password_ = input('비밀번호를 입력하세요:   ')
search = input('검색할 내용을 입력 하세요:   ')
search_count = int(input('몇건을 검색 할까요:   '))

아이디를 입력하세요:   
비밀번호를 입력하세요:   
검색할 내용을 입력 하세요:   
몇건을 검색 할까요:   1


In [2]:

chrome_path = 'C:\chromedriver\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
url = 'https://www.instagram.com/'
driver.get(url)
time.sleep(3)

## 한꺼번에 입력하거나 자주 실행하면 막힘.....ㅎㅎ
element = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').click()
for i in id_:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))

element = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').click()
for i in password_:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))
time.sleep(3)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(2)

## 자동로그인 설정
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

## 알림설정
driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
time.sleep(2)

driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[1]/div').click()
element = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')

for i in search:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))

time.sleep(2)
element.send_keys('\n')
time.sleep(3)
element.send_keys('\n')

time.sleep(6)
title_=[]
hash_tag = []
txt_ = []
img=[]

driver.find_element_by_class_name('_9AhH0').click()
time.sleep(3)
for i in range(search_count+1):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    try:
        title_text = soup.find('a',class_='O4GlU').get_text()
        title_.append(title_text)
    except:
        title_text = ''
        title_.append(title_text)

    txt_text = soup.select_one('div.C4VMK>span').get_text()
    txt_.append(txt_text)

    review_soup= soup.select('a.xil3i')
    a = ''
    for i in review_soup:
        a += i.get_text()+' '
    hash_tag.append(a)

    try:
        driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]').click()
    except:
        driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div').click()
    time.sleep(2)
    
## 자세히보기 x
driver.find_element_by_xpath('/html/body/div[6]/div[1]').click()
time.sleep(2)

## 사진 수집
while True:
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    list_a = soup.find('article','KC1QD').find_all('img')
    for i in list_a:
        list_url = i.get('src')
        img_url_.append(list_url)
    img_url = set(img_url_)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    if len(img_url)>= search_count:
        img_url = list(img_url)[:search_count]
        break


In [66]:
## 사진 url
a= 100
img_url_ = []
while True:
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    list_a = soup.find('article','KC1QD').find_all('img')
    for i in list_a:
        list_url = i.get('src')
        img_url_.append(list_url)
    img_url = set(img_url_)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    if len(img_url)>= a:
        img_url = list(img_url)[:50]
        break

In [126]:
for i in img_url:
    print(i)

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/273225861_390229286196039_760451436023459337_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=W_7nO9mtr4IAX-mFj6I&edm=AMKDjl4BAAAA&ccb=7-4&ig_cache_key=Mjc2NzMzNzMwOTk3OTc3NDY1NQ%3D%3D.2-ccb7-4&oh=00_AT__CoKT3Ptf30xN379XyqePiPMkuW2YRbUp6abR4pIRew&oe=620F6EBF&_nc_sid=1fe099
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/273523785_204533165179990_4681917586295743477_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=9-EL8zD86-wAX8gStmd&tn=wdaAbHRLG3Pkz4i7&edm=AMKDjl4BAAAA&ccb=7-4&ig_cache_key=Mjc2ODA4NTkzNDI2NTM3NDI0Mg%3D%3D.2-ccb7-4&oh=00_AT8d5SOvApNm7p5ZIO6bKg-15eW97UwSeCfgmmJXBMJJ0A&oe=6210657A&_nc_sid=1fe099
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/273369752_1626943780976299_1135823876727927454_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=107&_nc_ohc=E5LVQGTpLSsAX9PEbq7&edm=AMKDjl4BAAAA&ccb=7-4&ig_cache_key=Mjc2Njk4MDYwOTk1NDY0MjgyNQ%3D%3D.2-ccb7-4&oh=00_A

In [77]:
## 길이 맞추기
title_=[]
hash_tag = []
txt_ = []
img=[]

driver.find_element_by_class_name('_9AhH0').click()
time.sleep(3)
for i in range(10):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    try:
        title_text = soup.find('a',class_='O4GlU').get_text()
        title_.append(title_text)
    except:
        title_text = ''
        title_.append(title_text)

    txt_text = soup.select_one('div.C4VMK>span').get_text()
    txt_.append(txt_text)

    review_soup= soup.select('a.xil3i')
    a = ''
    for i in review_soup:
        a += i.get_text()+' '
    hash_tag.append(a)

    try:
        driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]').click()
    except:
        driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div').click()
    time.sleep(2)

C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_4436/1324819915.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('_9AhH0').click()
C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_4436/1324819915.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]').click()
C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_4436/1324819915.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div').click()


In [82]:
print(len(title_))
print(len(hash_tag))
print(len(txt_))

10
10
10


In [84]:
data_df = pd.DataFrame()

In [85]:
data_df['제목'] = title_
data_df['해쉬 태그'] = hash_tag
data_df['내용'] = txt_

,제목,해쉬 태그,내용
0,"Changwon, Korea",,Changwon City Boy 👤
1,웅천읍성,#Nikon #nikonz6 #ALLaboutNIKON #Web04 #changwo...,진해 웅천 읍성에서 만난 옹성옹성이란성문앞에 설치되는 시설물(성벽)인데 그 형상이 ...
2,,#창원맛집 #창원맛집추천 #창원돈까스맛집 #창원데이트 #창원핫플레이스 #창원핫플 #...,"( 협찬 )"" 창원에서 제일 맛있는 돈까스 찐-맛집 !! ""…….....…………....."
3,무진정,#함안 #무진정 #경남여행,무진장 추운 무진정#함안 #무진정 #경남여행
4,,#키친187 #창원맛집 #창원카페 #창원파스타 #창원데이트 #창원 #창원여행 #창원...,"..창원에는 거제와는 또다른 매력❤️아니 마력이라고 해야하나? ㅎㅎ.연인 데이트, ..."
5,창원 가로수길 영국집,#풍경 #풍경사진 #창원 #창원가로수길 #가로수길 #메타세콰이어길 #창원여행 #창원...,x Changwon🍁창원 가로수길🍁메타세콰이어 가로수길은 아직 가을이에요창원에 살지...
6,진해보타닉뮤지엄,#📷경상남도_창원 #진해보타닉뮤지엄 #보타닉뮤지엄 #창원가볼만한곳 #진해가볼만한곳 ...,"211230 #📷경상남도_창원사계절 가볼만한 수목원 ""진해보타닉뮤지엄""전구를 걸쳐서..."
7,창원,#여행중독 #창원가볼만한곳 #창원여행 #창원데이트 #창원여행코스추천 #창원맛집 #창...,.[🌊청량감 가득한 오션뷰와 새우들이 팔딱!팔딱!🌊]계획적인 여행도 좋지만 이런 즉...
8,저도연육교,#창원여행 #마산여행 #저도연육교 #창원야경 #마산야경 #후지필름_나의사진이야기 #...,따뜻한 캔커피가 생각나는 밤🌁..@7elevenkorea..#창원여행 #마산여행 #...
9,,#노을보러왔굴 #창원맛집 #창원데이트 #창원여행 #마산맛집 #마산여행 #김해근교,창원에 생겼다는 오션+노을뷰 끝판왕 굴구이집🦪탁 트인 파노라마 오션뷰에 웅장이 가슴...


In [ ]:
## 사진 마무리